### How to use get_gnd_state_probs_and_approx_ratio




a) get_gnd_state_probs_and_approx_ratio_simple_init(p, prt_details=1, state_feasible=None,**Ansatz_type1) supplies:  
    1) Create 3 adjacency dicts: Adjacency,Adjacency_constraint,Adjacency_feasible
    2) Create the quantum circuit of the ansatz_prog, with the opt_betagamma
    3) save run time by suppling a example of a feasible state
b)  def get_gnd_state_probs_and_approx_ratio(opt_betagamma,  prt_details=1) is the function returned by the above

In [1]:
#TWO AMBULANCE DISTANCE MINIMISATION - 

#################################### Create Adjacency for a qubo problem of two ambulances ####################################
import numpy as np 
#RC utility functions    # C:\Users\Robert\Anaconda3\envs\rigoct38_env\Lib\site-packages\pyaqc\RCModules\
from pyaqc.RCModules.PlatformConversion import print_QUBOdetails, CreateTwoAmbulanceAdjacencyV2
Ansatz_type ={}         # dict of arguments that describe the problem parameters
Ansatz_type['n_destinations'] = 4  
Ansatz_type['n_qubits'] = Ansatz_type['n_destinations'] * 4
Ansatz_type['n_ambulance'] = 2
gridWidth = Ansatz_type['n_destinations']
Ansatz_type['HammingWeightOfConstraint'] = Ansatz_type['n_destinations']
Ansatz_type['use_XYMixer_constraints'] =1   # remove most of the hard constraints from the problem Adjacency when using an XY mixer
Ansatz_type['ConstraintMultiplier'] = 10        # use 90 for X mixer
#'Adjacency' table of the whole problem
result = CreateTwoAmbulanceAdjacencyV2(    gridWidth,Ansatz_type['n_destinations'], Adddistance= 1, ConstraintMultiplier=Ansatz_type['ConstraintMultiplier'],use_XYMixer_constraints=Ansatz_type['use_XYMixer_constraints'])

Ansatz_type['Adjacency'] = result['qubo']
quboHybrid = result['quboHybrid']
n_qubits = result['n_qubits']
ConstraintMultiplier = result['ConstraintMultiplier']
max_distance = result['max_distance']
sum_distance = result['sum_distance']

# Adjacency that defines whether a given state is 'feasible'.This is needed to calculate the 'approximation ratio' arXiv:1904.09314v2 [quant-ph] 21 May 2020 
result_feas = CreateTwoAmbulanceAdjacencyV2(    gridWidth,Ansatz_type['n_destinations'], Adddistance= 0, ConstraintMultiplier=Ansatz_type['ConstraintMultiplier'],
use_XYMixer_constraints=0)
Ansatz_type['Adjacency_feasible']= result_feas['qubo']

#Adjacency of just the problems constraints
result_cst = CreateTwoAmbulanceAdjacencyV2(gridWidth,Ansatz_type['n_destinations'], Adddistance = 0,ConstraintMultiplier=Ansatz_type['ConstraintMultiplier']
                                            , use_XYMixer_constraints =Ansatz_type['use_XYMixer_constraints'])

Ansatz_type['Adjacency_constraint'] = result_cst['qubo']

select_qubo_model = 1  # This is automatically the case in the function CreateTwoAmbulanceAdjacencyV2
Width  = gridWidth
Height = Ansatz_type['n_destinations'] //gridWidth

if 1:
    filename = 'Twoambulances.txt'
    print('Qubo ', end='')
    print_QUBOdetails(result['qubo'],n_qubits,filename)
    print(Ansatz_type)

Qubo ADJACENCY MATRIX Twoambulances.txt  

Qubit  q0   q1   q2   q3   q4   q5   q6   q7   q8   q9  q10  q11  q12  q13  q14  q15 

  q0   0                   20                       1    4    9  

  q1                            20             1         1    4  

  q2                                 20        4    1         1  

  q3                                      20   9    4    1  

  q4                                                                    1    4    9  

  q5                                                               1         1    4  

  q6                                                               4    1         1  

  q7                                                               9    4    1  

{'n_destinations': 4, 'n_qubits': 16, 'n_ambulance': 2, 'HammingWeightOfConstraint': 4, 'use_XYMixer_constraints': 1, 'ConstraintMultiplier': 10, 'Adjacency': {(0, 0): 0, (0, 4): 20, (0, 9): 1, (0, 10): 4, (0, 11): 9, (1, 5): 20, (1, 8): 1, (1, 10): 1, (1, 11): 4,

In [2]:
#ONE AMBULANCE adjacency table
from pyaqc.RCModules.aqc_rigetti import get_gnd_state_probs_and_approx_ratio

from pyaqc.RCModules.PlatformConversion import print_QUBOdetails, CreateAmbulanceAdjacency,distance #   qc_all_env and rigoct38_env
import numpy as np 
import unicodedata as ud
betagreek =  ud.lookup('GREEK SMALL LETTER BETA')
gammagreek =  ud.lookup('GREEK SMALL LETTER gamma')

################  Set up type of problem to solve ################
Ansatz_type={}
Ansatz_type['n_ambulance'] = 1
Ansatz_type['n_destinations'] = 5
Ansatz_type['n_qubits'] = Ansatz_type['n_destinations']
Ansatz_type['HammingWeightOfConstraint']  =  Ansatz_type['n_qubits'] - 1

gridWidth = Ansatz_type['n_qubits'] 
Ansatz_type['ConstraintMultiplier'] = 14#200             # I have used 14 for 5 qubit with constraints and traditional mixer, 140 for q11, tim 200
# 1500 for q17
# 800 for q8
Ansatz_type['use_XYMixer_constraints'] =  0   # remove the constraints when using an XY mixer
select_qubo_model = 1   # we nearly always use this
################ Create 1) 'Adjacency' table of the problem, and 2) 'Adjacency_constraint' of the problems constraints 3) Adjacency_feasible ################
#'Adjacency' table of the whole problem

Ansatz_type['Adjacency'] = CreateAmbulanceAdjacency(   
    gridWidth,
    n_qubits = Ansatz_type['n_qubits'], 
    ConstraintMultiplier = Ansatz_type['ConstraintMultiplier'], 
    Adddistance = 1,
    remove_constraint= Ansatz_type['use_XYMixer_constraints'] , 
    HammingWeightOfConstraint= Ansatz_type['HammingWeightOfConstraint'] ,
    qubo_model=select_qubo_model)
#Adjacency of just the problems constraints

Ansatz_type['Adjacency_constraint'] = CreateAmbulanceAdjacency(   
    gridWidth,
    n_qubits = Ansatz_type['n_qubits'], 
    ConstraintMultiplier = Ansatz_type['ConstraintMultiplier'], 
    Adddistance = 0,
    remove_constraint= Ansatz_type['use_XYMixer_constraints'] , 
    HammingWeightOfConstraint= Ansatz_type['HammingWeightOfConstraint'] ,
    qubo_model=select_qubo_model)
# Adjacency that defines whether a given state is 'feasible'
Ansatz_type['Adjacency_feasible'] = CreateAmbulanceAdjacency(   
    gridWidth,
    n_qubits = Ansatz_type['n_qubits'], 
    ConstraintMultiplier = Ansatz_type['ConstraintMultiplier'], 
    Adddistance = 0,
    remove_constraint= 0, 
    HammingWeightOfConstraint= Ansatz_type['HammingWeightOfConstraint'] ,
    qubo_model=select_qubo_model)
################ ... and print ################
if select_qubo_model:print('Qubo ', end='')
else: print('Ising ', end='')
print_QUBOdetails(Ansatz_type['Adjacency'] ,Ansatz_type['n_qubits'], 'Ambulance problem')

Qubo ADJACENCY MATRIX Ambulance problem  

Qubit  q0   q1   q2   q3   q4 

  q0  -98   27   24   19   12 

  q1       -98   27   24   19 

  q2            -98   27   24 

  q3                 -98   27 

  q4                      -98 



In [3]:
'Angles that have been optimised for various problems, q5 to q19, One Amb, q16 two Amb'
q5p1TradXmHqq = [6.898, -0.022] # nm -252.8 #11.1% Energy_max =-20 , Energy_min = -40 when constraint are complied with and have zero energy contribution to the cost function
q5p1RmDi = [3.9924273766343394, 2.827450565142573] # -38.4 EV of problem   71.1% r_approx new = 92.1%
q5p3RmDi = [3.963075998397324, 5.397047029891928, 2.3765397005374207, 3.1643560454228994, -0.03195512120218456, 0.30846008251700574] # -39.941 0.991 35, 0.997 = r_approx (E_max =0)
q5p3TradXmHqq =  [-2.113639856606643, -3.002768750687589, -4.828135143532405, 0.01965435129056827, 2.0763823207676677, 1.4405028404001305] #0.0370 4 570, = r_approx (E_max =0)
#q8
q8p1Hw6_TRAD =  [3.737202, 0.063006] #-6995.312, 0.001041 0.0062  = prob_gnd_state, 0.6487  = approx_ratio, 0.0228  = prob_feasible_state

q11p1RmDi = [ 0.868, -0.02 ] #-1050.45 19.5%
q11p3RmDi   =[1.431421, 0.817087, 2.028664, 0.957856, 1.974961, 0.425763] # -1081.7 32.3=p_success, old approx=95.5, new approx = 93.3 
q11p3HW10_interp_TT_NM = [0.6286729798152555, 0.6094902517294243, 0.5605768275595322, -0.011444792772741205, -0.00425603944107247, -0.012365080443628575] #interp_TT -1067.7 37.0%
q11p3HW10_interp_NM = [0.6286897379372094, 0.6094509771046137, 0.5605711207713572, -0.011444854953217572, -0.0042559478813491616, -0.01236289525811517] #interp -1067.7 37.0%
q11p1TradXmH =  [2.4438652809143067, 1.5738661033630374]#-12623 1.0% CM140 new r_approx=43.8%
q11p3TradXmHqq =[-1.4926870960164018, -2.318859891143705, -5.881436314418812, 1.2582855789828138, 0.31574711088728313, 2.5490460457525597]  #-12293. 0.002 = gnd state prob,  0.955 = approx_ratio, CM140
q11p3TradXmH_interp_from_p1 =     [-0.6192225858511726, -0.3778830066802197, -7.7985735007499e-06, 1.573682853644343, 0.8154860817413727, -3.446929781736484e-05] #-12885.265 0.025271412390374358 1 300 
q11p3TradXmH_interp_TT_from_p1 =      [-0.37878142557194916, 8.906034487538886e-05, -0.6154068094481024, 1.57327276621201, 0.004972151376513632, -0.003760861752566373] #-13326.243 0.01261894177042601 0 312

#q16
q16p3XmH2AmbulanceJoint= [1.986272, 2.932675, 4.789764, 6.321789, 6.199014, 5.657889] # angles =845 -446 0.0338  = prob_gnd_state, 0.7016  = approx_ratio, 0.4202  = prob_feasible_state,
q16p3XmH2Ambulance4pos =[0.45953481430317844, 6.325713629903305, 0.23136688630355373, 0.03878415622311383, 2.86906157059098, 3.0133339008244002] #  july 2021: -407.2  = ansatz_EV 0.027 = gnd state prob,  0.279 = approx_ratio , 90 =CM new r_approx = 0.784, p_feas = 24.1

q16p1XmH2Ambulance4pos = [3.5477810538668466, 6.322074470994686] # -325.5 0.6%  lambda =90 =CM new r_approx = 0.710, old = 0.223, 200 angles
q16p1XmH2Ambulance1056joint = [2.740435, 0.004513] # -331.764, 0.004751, 207.0,lambda =90 =CM 0.0048  = prob_gnd_state, 0.6712  = approx_ratio, 0.0968  = prob_feasible_state
#q16p1XmH2Ambulance4pos = [-3.5477810538668466, 6.322074470994686] #'-' is the old def of X mixer
#'+3.58' is the new def of X mixer
q16p1Xmp3_from_p1_interp_TT = [3.5880406863876857, 0.23162758481044704, 0.0, 6.321721070536895, 0.016592338309179132, 0.000125]# -463 3.5% r_approx new = 72%, p_feas = 24%
q16p1Xmp4_from_p1_interp_TT = [3.5880406863876857, 0.23162758481044704, 0.0, 0.0, 6.321721070536895, 0.016592338309179132, 0.000128125, 0.000125] # -463.4 
#Different p1 start E= -332 frm q16p1XmH2Ambulance1056joint
q16p1Xmp3_from_p1_interp_TT = [2.7231464245792183, -0.22722010571361953, 1.1093618374268595e-05, 0.00442823877715909, 0.015793035940631316, -2.539460473422888e-06] #-480.48 0.0292  = prob_gnd_state, 0.6751  = approx_ratio, 0.5763  = prob_feasible_state

p1g1b1q16iter1000ang2002pi = [0.29906068699559607, 6.20089436681941] #15.2 #89 lambda =10 =CM 0.0324  = prob_gnd_state, 0.7457  = approx_ratio, 0.6525  = prob_feasible_state,
p2g1b1q16iter1000ang2002pi = [6.102801120961688, 0.3141269065682929, 0.10311645636232411, 6.092037367414478] # 13.59
p3g1b1q16iter2500ang1000pi = [5.846958363396367, 6.010501043321872, 0.04458020114064164, 6.344488601986301, 6.391955815676612, 3.587765610409201] # 8.196 0 29 532 ,0.0324  = prob_gnd_state, 0.7457  = approx_ratio, 0.6525  = prob_feasible_state,
p3g1b1_interp_from_p1 = [0.44684552046128023, 0.3619722948479162, 0.13474098580981728, 6.2328617349090045, 6.182524977574296, 6.146570633893404]  #interp  7.282 0.0646 1 418 p3 from p1,  0.334 = approx_ratio

p9g1b1q16iter2500pi = [0.08835910318366946, -0.3429756090990961, -0.35202631065024437, 0.06891975915178547, -0.260860565868011, -0.2103650424349861, -0.03731185998588227, 0.15358711272928655, -0.01221615403014623, 6.2272157395370265, 0.0987956485159821, 0.0985279301327055, -0.18539244563333002, 0.315360645315927, 0.1312015697718394, 0.025510671415913545, -0.03312621219095406, -0.01373866328274774] #6.236 interp_TT 10%

q17p1TradH100angles = [-5.556807424465457, -0.0041934427604425565] #-377379.9,  0.007 = gnd state prob,  0.894 = approx_ratio,0.12758  =prob_feasible_state,  # 0.367= approx_ratio new 
q19p1RmDi = [0.982341, 3.147795]  #-9939.867=EV_min (12 repeats), 0.014, 3.0,  0.014 = gnd state prob,  0.969 = approx_ratio, 79.2 = approx_ratio new
print('Angles set')

Angles set


In [4]:
 ################# Cell calculates prob_gnd_state, approx_ratio, prob_feasible_state  FROM...
 # opt_betagamma,n_qubits,use_XYMixer_constraints,n_ambulance,Ansatz_type,Adjacency,Adjacency_constraint,Adjacency_feasible   ############################
from pyaqc.RCModules.aqc_rigetti import get_prog_init, get_mixer, get_QAOA_circuit, get_gnd_state_probs_and_approx_ratio_simple_init
#from pyaqc.RCModules.aqc_rigetti import get_mixer,get_prog_init, get_approx_ratio_init,get_gnd_state_probs_and_approx_ratio,ansatz_prog_init,Adjacency_qubo_to_Regetti,get_QAOA_circuit#,get_gnd_state_probs_and_approx_ratio_simple_init

opt_betagamma = q5p1TradXmHqq# p1g1b1q16iter1000ang2002pi # p1g1b1q16iter1000ang2002pi
p = len(opt_betagamma)//2

#Supply a 'Adjacency_feasible', whose energy in 'state_feasible' can be used to identify whether a given state is a feasible state.
# if state_feasible==None then all 2^n states of 'Adjacency_feasible' will be searched and the lowest energy will define a feasible state.
n_q = Ansatz_type['n_qubits']
if n_q == 5 :state_feasible = '11011'
if n_q == 8 :state_feasible = '11100111'
if n_q == 11 :state_feasible = '11111011111'
if n_q==16:state_feasible='1000 0001 1100 0011'        #|qn...0> |start,start,destination>
if n_q == 17 :state_feasible = '11111111011111111'
if n_q == 19 :state_feasible = '1111111110111111111'
if 0: state_feasible=None

# Initiate function to be used
# simpler call that creates the QAOA Ansatz from the Adjacency
############### SIMPLE initiation ###############
if 1: get_gnd_state_probs_and_approx_ratio = get_gnd_state_probs_and_approx_ratio_simple_init(p, prt_details=1, state_feasible=state_feasible,**Ansatz_type)
#= get_gnd_state_probs_and_approx_ratio_simple_init(n_qubits, n_destinations,n_ambulance,p,use_XYMixer_constraints,HammingWeightOfConstraint,Adjacency,Adjacency_constraint,Adjacency_feasible, prt_details=1, state_feasible=state_feasible,**Ansatz_type)

elif 1: 
    ############### SHOW DETAIL of  initiation ###############
    prog_init,MyMixerHam,ListPauli_termsMy,SumPauli_termsMy,ansatz_prog = get_QAOA_circuit(p, prt_details=0, state_feasible=None,**Ansatz_type)
    
    #get_QAOA_circuit(n_qubits, n_destinations,p,Adjacency_constraint,Adjacency_feasible, prt_details=1, state_feasible=state_feasible)
    get_gnd_state_probs_and_approx_ratio = get_approx_ratio_init(prog_init,MyMixerHam,ListPauli_termsMy,SumPauli_termsMy,ansatz_prog,Ansatz_type['n_qubits'], Ansatz_type['n_destinations'],p,Ansatz_type['Adjacency_constraint'],Ansatz_type['Adjacency_feasible'], prt_details=1, state_feasible=state_feasible)
    

prob_gnd_state, approx_ratio, prob_feasible_state = get_gnd_state_probs_and_approx_ratio(opt_betagamma,prt_details=1)
print('\nReturned by get_gnd_state_probs_and_approx_ratio()  ;\n'
    '%4.4f'%prob_gnd_state, ' = prob_gnd_state,\n', '%4.4f'%approx_ratio, ' = approx_ratio,\n', '%4.4f'%prob_feasible_state, ' = prob_feasible_state,\n')
#print(Ansatz_type)


Returned by get_gnd_state_probs_and_approx_ratio()  ;
0.1107  = prob_gnd_state,
 0.5843  = approx_ratio,
 0.4389  = prob_feasible_state,



In [5]:
from pyaqc.RCModules.aqc_rigetti import ansatz_EV_init,get_QAOA_circuit
opt_betagamma = p1g1b1q16iter1000ang2002pi
p = len(opt_betagamma)//2

prog_init,MyMixerHam,ListPauli_terms,SumPauli_terms,ansatz_prog = get_QAOA_circuit(p, prt_details=0, state_feasible=state_feasible,**Ansatz_type)
ansatz_EV = ansatz_EV_init(prog_init,  ListPauli_terms, Ansatz_type['n_qubits'] ,Ansatz_type['n_destinations'] ,   p, MyMixerHam)
EV =  ansatz_EV(opt_betagamma)
if 1:
    print('%3.4f'%EV, ' = ansatz_EV(',opt_betagamma,')')    
    print(Ansatz_type)

-170.5575  = ansatz_EV( [0.29906068699559607, 6.20089436681941] )
{'n_ambulance': 1, 'n_destinations': 5, 'n_qubits': 5, 'HammingWeightOfConstraint': 4, 'ConstraintMultiplier': 14, 'use_XYMixer_constraints': 0, 'Adjacency': {(0, 0): -98, (0, 1): 27, (0, 2): 24, (0, 3): 19, (0, 4): 12, (1, 1): -98, (1, 2): 27, (1, 3): 24, (1, 4): 19, (2, 2): -98, (2, 3): 27, (2, 4): 24, (3, 3): -98, (3, 4): 27, (4, 4): -98}, 'Adjacency_constraint': {(0, 0): -98, (0, 1): 28, (0, 2): 28, (0, 3): 28, (0, 4): 28, (1, 1): -98, (1, 2): 28, (1, 3): 28, (1, 4): 28, (2, 2): -98, (2, 3): 28, (2, 4): 28, (3, 3): -98, (3, 4): 28, (4, 4): -98}, 'Adjacency_feasible': {(0, 0): -98, (0, 1): 28, (0, 2): 28, (0, 3): 28, (0, 4): 28, (1, 1): -98, (1, 2): 28, (1, 3): 28, (1, 4): 28, (2, 2): -98, (2, 3): 28, (2, 4): 28, (3, 3): -98, (3, 4): 28, (4, 4): -98}}


In [6]:
# Run the same problem as above but with a different p and angles
opt_betagamma = [3.761008479521208, 3.236440661053116, 0.06304026346071374, 0.06678436574431272]
prob_gnd_state, approx_ratio, prob_feasible_state = get_gnd_state_probs_and_approx_ratio(opt_betagamma,prt_details=1)
print('%4.4f'%prob_gnd_state, ' = prob_gnd_state,\n', '%4.4f'%approx_ratio, ' = approx_ratio,\n', '%4.4f'%prob_feasible_state, ' = prob_feasible_state,\n')
#print(get_gnd_state_probs_and_approx_ratio.__doc__)

0.0705  = prob_gnd_state,
 0.5227  = approx_ratio,
 0.3076  = prob_feasible_state,

